# Reading combine limits

The CMS Higgs Physics Analysis Group supports a widely used tool called [combine](https://cms-hcomb.gitbooks.io/combine/content/). `hepdata_lib` has the capability to directly import the limits from the ntuple created, see e.g. [Asymptotic Frequentists Limits](https://cms-hcomb.gitbooks.io/combine/content/part3/commonstatsmethods.html#asymptotic-frequentist-limits).

As explained in the [Getting started notebook](Getting_started.ipynb), a `Submission` needs to exist or be created. Here, we'll just create one without any additional information:

In [1]:
from hepdata_lib import Submission
submission = Submission()

Welcome to JupyROOT 6.28/06


The limit plot will again be a `Table`, in the example Figure 6a from page 14 of the publication. Let's add all this, some more details as well as the actual plot (for thumbnail creation) to the `Table`:

In [2]:
from hepdata_lib import Table
table = Table("Figure 6a")
table.description = "Exclusion limits on the product of the production cross section and the branching fraction for a new spin-2 resonance decaying to WW, as a function of the resonance mass hypothesis."

table.location = "Data from Figure 6a, located on page 14."
table.keywords["observables"] = ["SIG"]
table.keywords["reactions"] = ["P P --> GRAVITON --> W+ W-"]

table.add_image("example_inputs/Figure_006-a.pdf")

Now we can read in the `combine` ntuple, which in our case has the name `WWmerged.higgsCombineTest.Asymptotic.root`, and contains the upper limits on the production cross section of a resonance as a function of the mass of the resonance ("bulk graviton"). The file is read in with the `RootFileReader` using `read_limit_tree()`:

In [3]:
from hepdata_lib import RootFileReader
reader = RootFileReader("example_inputs/WWmerged.higgsCombineTest.Asymptotic.root")
data = reader.read_limit_tree()
print(data)

[[1.00000000e+03 1.82468891e-02 2.45227814e-02 3.55224609e-02
  5.20889647e-02 7.50288591e-02 2.16936472e-02]
 [1.05000000e+03 1.50303841e-02 2.02972870e-02 2.89306641e-02
  4.24229726e-02 6.07501119e-02 2.36925946e-02]
 [1.10000000e+03 1.26993656e-02 1.71843916e-02 2.45361328e-02
  3.55877690e-02 5.09835742e-02 2.60327992e-02]
 [1.15000000e+03 1.08987093e-02 1.47477984e-02 2.10571289e-02
  3.07096094e-02 4.38561961e-02 2.58909682e-02]
 [1.20000000e+03 9.50872898e-03 1.28669199e-02 1.83715820e-02
  2.67930217e-02 3.82629447e-02 2.56530152e-02]
 [1.25000000e+03 8.37147236e-03 1.13280192e-02 1.61743164e-02
  2.37174835e-02 3.37647162e-02 2.82829272e-02]
 [1.30000000e+03 7.48693943e-03 1.01310965e-02 1.44653320e-02
  2.10961662e-02 3.01272999e-02 2.87586050e-02]
 [1.35000000e+03 6.72876835e-03 9.10516270e-03 1.30004883e-02
  1.89598463e-02 2.70764343e-02 2.79213802e-02]
 [1.40000000e+03 6.04957342e-03 8.18609633e-03 1.16882324e-02
  1.70460586e-02 2.43433677e-02 2.63606315e-02]
 [1.450000

The `RootFileReader` returned an array of tuples, where each tuple is for a mass point, here starting at 1000 (GeV). If you have one file per mass point, you can easily chain together the ntuples in a loop. If you have text files, use `np.loadtxt("limits.txt", skiprows=0)` as in the [Getting started notebook](Getting_started.ipynb).

Let's define the variables and uncertainties that are in the ntuple. The format, i.e. their position in the tuple (observed, expected, ±1/±2 sigma variations) is a standard in `combine`. Each `Uncertainty` is associated to a `Variable` by adding it using `add_uncertainty()`:

In [4]:
from hepdata_lib import Variable, Uncertainty
d = Variable("Bulk graviton mass", is_independent=True, is_binned=False, units="GeV")
d.values = data[:,0]

obs = Variable("Cross section upper limit at 95% CL", is_independent=False, is_binned=False, units="pb")
obs.values = data[:,6]
obs.add_qualifier("Limit", "Observed")
obs.add_qualifier("SQRT(S)", 13, "TeV")
obs.add_qualifier("LUMINOSITY", 35.9, "fb$^{-1}$")

exp = Variable("Cross section upper limit at 95% CL", is_independent=False, is_binned=False, units="pb")
exp.values = data[:,3]
exp.add_qualifier("Limit", "Expected")
exp.add_qualifier("SQRT(S)", 13, "TeV")
exp.add_qualifier("LUMINOSITY", 35.9, "fb$^{-1}$")

# +/- 1 sigma
unc_1s = Uncertainty("1 s.d.", is_symmetric=False)
unc_1s.set_values_from_intervals(zip(data[:,2], data[:,4]), nominal=exp.values)
exp.add_uncertainty(unc_1s)

# +/- 2 sigma
unc_2s = Uncertainty("2 s.d.", is_symmetric=False)
unc_2s.set_values_from_intervals(zip(data[:,1], data[:,5]), nominal=exp.values)
exp.add_uncertainty(unc_2s)

This is it, all that needs to be done is to add the variables to the `Table` and the `Table` to the `Submission`, and create the files. Again, please refer to the [Getting started notebook](Getting_started.ipynb) for a complete example.

In [5]:
table.add_variable(d)
table.add_variable(obs)
table.add_variable(exp)
table.add_additional_resource("ROOT file", "example_inputs/WWmerged.higgsCombineTest.Asymptotic.root", copy_file=True)  # optional
submission.add_table(table)
submission.create_files("example_output",remove_old=True)

In [6]:
!ls example_output

Figure_006-a.png
WWmerged.higgsCombineTest.Asymptotic.root
figure_6a.yaml
submission.yaml
thumb_Figure_006-a.png


In [7]:
!head example_output/figure_6a.yaml

dependent_variables:
- header:
    name: Cross section upper limit at 95% CL
    units: pb
  qualifiers:
  - name: Limit
    value: Observed
  - name: SQRT(S)
    units: TeV
    value: 13
